# 算法 5.1（信息增益的算法）
- 测试数据：例5.2，表5.1
- 结果与书中结果基本一致

In [1]:
import pdb

import numpy as np
import pandas as pd
from IPython.display import display

# 表5.1 贷款申请样本数据表
loan_application = [
    (1, '青年', '否', '否', '一般', '否'),
    (2, '青年', '否', '否', '好', '否'),
    (3, '青年', '是', '否', '好', '是'),
    (4, '青年', '是', '是', '一般', '是'),
    (5, '青年', '否', '否', '一般', '否'),
    (6, '中年', '否', '否', '一般', '否'),
    (7, '中年', '否', '否', '好', '否'),
    (8, '中年', '是', '是', '好', '是'),
    (9, '中年', '否', '是', '非常好', '是'),
    (10, '中年', '否', '是', '非常好', '是'),
    (11, '老年', '否', '是', '非常好', '是'),
    (12, '老年', '否', '是', '好', '是'),
    (13, '老年', '是', '否', '好', '是'),
    (14, '老年', '是', '否', '非常好', '是'),
    (15, '老年', '否', '否', '一般', '否')
]

df = pd.DataFrame(loan_application, columns=['ID', '年龄', '有工作', '有自己的房子', '借贷情况', '类别'])
# display(df)

# 将类型转换为int类型，使用100、101、10、11、12等数字，而不是0、1、2等是为了便于在计算过程中区分各个类别
df['类别'] = df['类别'].map({'否': 100, '是': 101}).astype(int)
df['年龄'] = df['年龄'].map({'青年': 10, '中年': 11, '老年': 12}).astype(int)
df['有工作'] = df['有工作'].map({'否': 20, '是': 21}).astype(int)
df['有自己的房子'] = df['有自己的房子'].map({'否': 30, '是': 31}).astype(int)
df['借贷情况'] = df['借贷情况'].map({'一般': 40, '好': 41, '非常好': 42})
# display(df)

x_data = df[['年龄', '有工作', '有自己的房子', '借贷情况']].as_matrix()
y_data = df['类别'].as_matrix()

print(x_data)
print(y_data)
print(x_data.shape)
print(y_data.shape)

[[10 20 30 40]
 [10 20 30 41]
 [10 21 30 41]
 [10 21 31 40]
 [10 20 30 40]
 [11 20 30 40]
 [11 20 30 41]
 [11 21 31 41]
 [11 20 31 42]
 [11 20 31 42]
 [12 20 31 42]
 [12 20 31 41]
 [12 21 30 41]
 [12 21 30 42]
 [12 20 30 40]]
[100 100 101 101 100 100 100 101 101 101 101 101 101 101 100]
(15, 4)
(15,)


In [2]:
# （1）经验熵H(D)
import math

def empirical_entropy(labels):
    # labels：int类型的list
    lenght = len(labels)
    class_names = set(labels)
    # 用字典来统计各个值出现的次数
    class_group = {}
    # 初始化字典，存储各个类别的数量都为0
    for c in class_names:
        class_group[c] = list(labels).count(c)
    # 汇总各个类别，【p62，公式5.7】
#     entropies = []
#     for g in class_group.values():
#         entropies.append(- g / lenght * math.log2(g / lenght))
#     entropy = sum(entropies)
    entropy = sum([- g / lenght * math.log2(g / lenght) for g in class_group.values()]) #这一行代码等同于前面4行
    return entropy

entropy = empirical_entropy(y_data)
print('（1）经验熵H(D) = %.3f' % entropy)

（1）经验熵H(D) = 0.971


In [3]:
# （2）条件经验熵H(D|A)

def conditional_empirical_entropy(feature, labels):
    # feature：单个特征的int类型的list，int是map之后的类别，支持多个特征取条件经验熵
    # labels：int类型的list，int是map之后的类别
    lenght = len(labels)
    classes = set(feature)
    # 按feature对数据进行分组
    feature_group = {}
    for c in classes:
        indexes = [i for i in range(len(feature)) if feature[i] == c]
        feature_group[c] = indexes
#     print(feature_group)
    entropies = []
    # 【p62，公式5.8】
    for indexes in feature_group.values():
        di_d = len(indexes) / lenght
        entropies.append(di_d*empirical_entropy(labels[indexes]))
    entropy = sum(entropies)
    return entropy

entropy = conditional_empirical_entropy(x_data[:, 0], y_data)
print('（2）条件经验熵H(D|A) = %.3f' % entropy)

（2）条件经验熵H(D|A) = 0.888


In [4]:
# （3）信息增益g(D,A)
def information_gain(feature, labels):
    # 【p62，公式5.9】
    return empirical_entropy(labels) - conditional_empirical_entropy(feature, labels)

gain = information_gain(x_data[:, 0], y_data)
print('（3）信息增益g(D,A) = %.3f' % gain)

（3）信息增益g(D,A) = 0.083


In [5]:
# 计算各个特征的信息增益
for i in range(x_data.shape[1]):
    print(information_gain(x_data[:, i], y_data))

def get_max_information_gain(features, labels):
    gains = [information_gain(features[:, i], y_data) for i in range(features.shape[1])]
    sorted_indexes = np.argsort(gains)
#     print(sorted_indexes)
    return sorted_indexes[-1], gains[sorted_indexes[-1]]

max_feature_index, max_feature_gain = get_max_information_gain(x_data, y_data)
print('信息增益最大的index是：%s，对应特征是：A(%s)，信息增益是：%.3f' % (max_feature_index, max_feature_index + 1, max_feature_gain))

0.08300749985576883
0.32365019815155627
0.4199730940219749
0.36298956253708536
信息增益最大的index是：2，对应特征是：A(3)，信息增益是：0.420
